<a href="https://colab.research.google.com/github/Waleed-Alfaris/Predict-NBA-Results/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import math

#transformations
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [ ]:
games_src = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/games.csv')
games_src.dropna(inplace=True)
games_src.drop(columns=['GAME_STATUS_TEXT', 'TEAM_ID_home', 'TEAM_ID_away'], inplace=True) #drop redundant and unnecessary columns
games_src['GAME_DATE_EST'] = pd.to_datetime(games_src['GAME_DATE_EST'])
games_src = games_src.sort_values(by='GAME_DATE_EST')
games_src.reset_index(drop=True, inplace=True)
games_src = games_src.head(23372) #drop the 50, 2020 games (not enough data)
games_src[games_src['GAME_ID'] == 20300001]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
15,2003-10-28,20300001,1610612755,1610612748,2003,89.0,0.44,0.533,0.35,25.0,39.0,74.0,0.408,0.824,0.25,16.0,44.0,1


In [ ]:
#train test split
games_src_train = games_src.head(int(len(games_src) * 0.9975) + 1)
games_src_test = games_src.tail(int(len(games_src) * 0.0025))

len(games_src_train), len(games_src_test)

(23314, 58)

In [ ]:
#save to csv
games_src_train.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/games_train.csv')
games_src_test.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/games_test.csv')

In [ ]:
games_src_train.head()

NameError: ignored

# ELO

In [ ]:
#Home and road team win probabilities implied by Elo ratings and home court adjustment
def win_prob(h_elo, a_elo, h_court_advantage):
    home = math.pow(10, h_elo/400)
    road = math.pow(10, a_elo/400)
    adj = math.pow(10, h_court_advantage/400) 

    denom = road + adj*home
    h_prob = adj*home / denom
    a_prob = road / denom 

    return h_prob, a_prob

#odds the home team will win based on elo ratings and home court advantage
def home_odds(h_elo, a_elo, h_court_advantage):
    home = math.pow(10, h_elo/400)
    road = math.pow(10, a_elo/400)
    adj = math.pow(10, h_court_advantage/400)
    return adj*home/road

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, delta_elo):
    k = 20
    if MOV>0:
        mult=(MOV+3)**(0.8)/(7.5+0.006*(delta_elo))
    else:
        mult=(-MOV+3)**(0.8)/(7.5+0.006*(-delta_elo))
    return k*mult


#updates the home and away teams elo ratings after a game 
def update_elo(h_score, a_score, h_elo, a_elo, home_court_advantage):
    h_prob, a_prob = win_prob(h_elo, a_elo, home_court_advantage) 

    if (h_score - a_score > 0):
        h_win = 1 
        a_win = 0 
    else:
        h_win = 0 
        a_win = 1 

    k = elo_k(h_score - a_score, h_elo - a_elo)

    new_h_elo = h_elo + k * (h_win - h_prob) 
    new_a_elo = a_elo + k * (a_win - a_prob)

    return new_h_elo, new_a_elo


#takes into account prev season elo
def prev_elo(team, date, season, team_stats, elo_df):
    prev_game = team_stats[team_stats['GAME_DATE_EST'] < date][(team_stats['HOME_TEAM_ID'] == team) | (team_stats['VISITOR_TEAM_ID'] == team)].sort_values(by = 'GAME_DATE_EST').tail(1).iloc[0] 

    if team == prev_game['HOME_TEAM_ID']:
        elo_rating = elo_df[elo_df['GAME_ID'] == prev_game['GAME_ID']]['H_Post_Game_ELO'].values[0]
    else:
        elo_rating = elo_df[elo_df['GAME_ID'] == prev_game['GAME_ID']]['A_Post_Game_ELO'].values[0]
  
    if prev_game['SEASON'] != season:
        return (0.75 * elo_rating) + (0.25 * 1505)
    else:
        return elo_rating

In [ ]:
elo_df = pd.DataFrame(columns=['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'H_Pre_Game_ELO', 'A_Pre_Game_ELO', 'H_Post_Game_ELO', 'A_Post_Game_ELO'])

for idx, row in games_src_train.iterrows():
  game_id = row['GAME_ID']
  date = row['GAME_DATE_EST']
  season = row['SEASON']
  h_team = row['HOME_TEAM_ID']
  a_team = row['VISITOR_TEAM_ID']
  h_score = row['PTS_home']
  a_score = row['PTS_away']
  
  #check if team has already been found. If not, ELO is 1500, if so, find its most current ELO rating
  if (h_team not in elo_df['HOME_TEAM_ID'].values) and (h_team not in elo_df['VISITOR_TEAM_ID'].values):
    h_pregame_elo = 1500
  else:
    h_pregame_elo = prev_elo(h_team, date, season, games_src_train, elo_df)
      
  if (a_team not in elo_df['HOME_TEAM_ID'].values and a_team not in elo_df['VISITOR_TEAM_ID'].values):
    a_pregame_elo = 1500
  else:
    a_pregame_elo = prev_elo(a_team, date, season, games_src_train, elo_df)
      
  #calculate new ELO using game results
  h_postgame_elo, a_postgame_elo = update_elo(h_score, a_score, h_pregame_elo, a_pregame_elo, 69)
  
  #create new row and append to elo_df
  elo_df_row = dict(GAME_ID = game_id,
                    GAME_DATE_EST = date,
                    HOME_TEAM_ID = h_team,
                    VISITOR_TEAM_ID = a_team,
                    H_Pre_Game_ELO = h_pregame_elo,
                    A_Pre_Game_ELO = a_pregame_elo,
                    H_Post_Game_ELO = h_postgame_elo,
                    A_Post_Game_ELO = a_postgame_elo)
  
  elo_df = elo_df.append(elo_df_row, ignore_index=True)

#remove post game ELO, which was only used to calculate new ELO  
elo_df.drop(columns=['H_Post_Game_ELO', 'A_Post_Game_ELO'], inplace=True)
elo_df.rename(columns={'A_Pre_Game_ELO':'A_ELO', 'H_Pre_Game_ELO':'H_ELO'}, inplace=True)

In [ ]:
#save to csv
elo_df.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/ELO.csv', index=False)

In [ ]:
#using the ELO dataframe, make a new dataframe with each teams most recent ELO
#this will be used to append ELO ratings to X_test
team_elo_df = pd.DataFrame(columns=['TEAM', 'ELO'])

for team in elo_df['HOME_TEAM_ID'].unique():
  team_df = elo_df[(elo_df['HOME_TEAM_ID'] == team) | (elo_df['VISITOR_TEAM_ID'] == team)].sort_values(by='GAME_DATE_EST').tail(1).iloc[0]
  
  h_team = team_df['HOME_TEAM_ID']
  a_team = team_df['VISITOR_TEAM_ID']
  h_elo = team_df['H_ELO']
  a_elo = team_df['A_ELO']

  if team == h_team:
    elo = h_elo
  else:
    elo = a_elo

  team_elo_df_row = dict(TEAM = team,
                         ELO = elo)

  team_elo_df = team_elo_df.append(team_elo_df_row, ignore_index=True)

In [ ]:
#save to csv
team_elo_df.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Team_ELO.csv', index=False)

# Recent Performance
- last 10 games
- last 5 games
- last 3 games
- season

In [ ]:
#function that will return the avergae stats from the last n_games
def get_recent_stats(team, date, games_df, n_games):
  #first take only the rows that have game_dates prior to row being checked
  #second, strip all rows that do not contain the team in question and take only the last 'n_games' rows
  prev_games = games_df[games_df['GAME_DATE_EST'] < date]
  team_df = prev_games[(prev_games['HOME_TEAM_ID'] == team) | (prev_games['VISITOR_TEAM_ID'] == team)].sort_values(by='GAME_DATE_EST').tail(n_games)

  #split home and away team stats
  h_df = team_df.iloc[:, [2,5,6,7,8,9,10]]
  a_df = team_df.iloc[:, [3,11,12,13,14,15,16]]

  #drop all rows that do not contain the team in question
  h_df = h_df[h_df['HOME_TEAM_ID'] == team]
  a_df = a_df[a_df['VISITOR_TEAM_ID'] == team]

  #rename columns so we can merge the two dataframes
  h_df.rename(columns={'HOME_TEAM_ID':'TEAM', 
                       'PTS_home':'SCORE',
                       'FG_PCT_home':'FG_PCT',
                       'FT_PCT_home':'FT_PCT',
                       'FG3_PCT_home':'FG3_PCT',
                       'AST_home':'AST',
                       'REB_home':'REB'}, inplace=True)
  
  a_df.rename(columns={'VISITOR_TEAM_ID':'TEAM', 
                       'PTS_away':'SCORE',
                       'FG_PCT_away':'FG_PCT',
                       'FT_PCT_away':'FT_PCT',
                       'FG3_PCT_away':'FG3_PCT',
                       'AST_away':'AST',
                       'REB_away':'REB'}, inplace=True)

  #combine the two and then return the mean values for each stat
  recent_df = pd.concat([h_df, a_df])

  return recent_df.mean()

In [ ]:
#generate the final df with the columns required
recent_stats_df = pd.DataFrame(columns=['GAME_DATE_EST',
                                        'GAME_ID',
                                        'SEASON',
                                        'HOME_TEAM_ID',
                                        'VISITOR_TEAM_ID',
                                        'H_LAST_3_PTS',
                                        'A_LAST_3_PTS',
                                        'H_LAST_3_FG_PCT',
                                        'A_LAST_3_FG_PCT',
                                        'H_LAST_3_FT_PCT',
                                        'A_LAST_3_FT_PCT',
                                        'H_LAST_3_FG3_PCT',
                                        'A_LAST_3_FG3_PCT',
                                        'H_LAST_3_AST',
                                        'A_LAST_3_AST',
                                        'H_LAST_3_REB',
                                        'A_LAST_3_REB',
                                        'H_LAST_5_PTS',
                                        'A_LAST_5_PTS',
                                        'H_LAST_5_FG_PCT',
                                        'A_LAST_5_FG_PCT',
                                        'H_LAST_5_FT_PCT',
                                        'A_LAST_5_FT_PCT',
                                        'H_LAST_5_FG3_PCT',
                                        'A_LAST_5_FG3_PCT',
                                        'H_LAST_5_AST',
                                        'A_LAST_5_AST',
                                        'H_LAST_5_REB',
                                        'A_LAST_5_REB',
                                        'H_LAST_10_PTS',
                                        'A_LAST_10_PTS',
                                        'H_LAST_10_FG_PCT',
                                        'A_LAST_10_FG_PCT',
                                        'H_LAST_10_FT_PCT',
                                        'A_LAST_10_FT_PCT',
                                        'H_LAST_10_FG3_PCT',
                                        'A_LAST_10_FG3_PCT',
                                        'H_LAST_10_AST',
                                        'A_LAST_10_AST',
                                        'H_LAST_10_REB',
                                        'A_LAST_10_REB'])

#iterate through every row, store the necessary data then call the get_recent_stats function with the stored data
for idx, row in games_src_train.iterrows():
  h_team = row['HOME_TEAM_ID']
  a_team = row['VISITOR_TEAM_ID']
  date = row['GAME_DATE_EST']
  game_id = row['GAME_ID']
  season = row['SEASON']

  #last 3 games
  h_last_3 = get_recent_stats(h_team, date, games_src_train, 3)
  a_last_3 = get_recent_stats(a_team, date, games_src_train, 3)

  #last 5 games
  h_last_5 = get_recent_stats(h_team, date, games_src_train, 5)
  a_last_5 = get_recent_stats(a_team, date, games_src_train, 5)

  #last 10 games
  h_last_10 = get_recent_stats(h_team, date, games_src_train, 10)
  a_last_10 = get_recent_stats(a_team, date, games_src_train, 10)

  #generate new row and then append to df
  recent_stats_df_row = dict(GAME_DATE_EST = date,
                            GAME_ID = game_id,
                            SEASON = season,
                            HOME_TEAM_ID = h_team,
                            VISITOR_TEAM_ID = a_team,
                            H_LAST_3_PTS = h_last_3['SCORE'],
                            H_LAST_3_FG_PCT = h_last_3['FG_PCT'],
                            H_LAST_3_FT_PCT = h_last_3['FT_PCT'],
                            H_LAST_3_FG3_PCT = h_last_3['FG3_PCT'],
                            H_LAST_3_AST = h_last_3['AST'],
                            H_LAST_3_REB = h_last_3['REB'],
                            A_LAST_3_PTS = a_last_3['SCORE'],
                            A_LAST_3_FG_PCT = a_last_3['FG_PCT'],
                            A_LAST_3_FT_PCT = a_last_3['FT_PCT'],
                            A_LAST_3_FG3_PCT = a_last_3['FG3_PCT'],
                            A_LAST_3_AST = a_last_3['AST'],
                            A_LAST_3_REB = a_last_3['REB'],
                            H_LAST_5_PTS = h_last_5['SCORE'],
                            H_LAST_5_FG_PCT = h_last_5['FG_PCT'],
                            H_LAST_5_FT_PCT = h_last_5['FT_PCT'],
                            H_LAST_5_FG3_PCT = h_last_5['FG3_PCT'],
                            H_LAST_5_AST = h_last_5['AST'],
                            H_LAST_5_REB = h_last_5['REB'],
                            A_LAST_5_PTS = a_last_5['SCORE'],
                            A_LAST_5_FG_PCT = a_last_5['FG_PCT'],
                            A_LAST_5_FT_PCT = a_last_5['FT_PCT'],
                            A_LAST_5_FG3_PCT = a_last_5['FG3_PCT'],
                            A_LAST_5_AST = a_last_5['AST'],
                            A_LAST_5_REB = a_last_5['REB'],
                            H_LAST_10_PTS = h_last_10['SCORE'],
                            H_LAST_10_FG_PCT = h_last_10['FG_PCT'],
                            H_LAST_10_FT_PCT = h_last_10['FT_PCT'],
                            H_LAST_10_FG3_PCT = h_last_10['FG3_PCT'],
                            H_LAST_10_AST = h_last_10['AST'],
                            H_LAST_10_REB = h_last_10['REB'],
                            A_LAST_10_PTS = a_last_10['SCORE'],
                            A_LAST_10_FG_PCT = a_last_10['FG_PCT'],
                            A_LAST_10_FT_PCT = a_last_10['FT_PCT'],
                            A_LAST_10_FG3_PCT = a_last_10['FG3_PCT'],
                            A_LAST_10_AST = a_last_10['AST'],
                            A_LAST_10_REB = a_last_10['REB'])

  recent_stats_df = recent_stats_df.append(recent_stats_df_row, ignore_index=True)

In [ ]:
#save to csv
recent_stats_df.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Recent_Stats.csv', index=False)

In [ ]:
#same as before with the ELO df, we will generate a new df that stores the most recent average stats for each team
team_recent_stats_df = pd.DataFrame(columns=['TEAM',
                                             'LAST_3_PTS',
                                             'LAST_3_FG_PCT',
                                             'LAST_3_FT_PCT',
                                             'LAST_3_FG3_PCT',
                                             'LAST_3_AST',
                                             'LAST_3_REB',
                                             'LAST_5_PTS',
                                             'LAST_5_FG_PCT',
                                             'LAST_5_FT_PCT',
                                             'LAST_5_FG3_PCT',
                                             'LAST_5_AST',
                                             'LAST_5_REB',
                                             'LAST_10_PTS',
                                             'LAST_10_FG_PCT',
                                             'LAST_10_FT_PCT',
                                             'LAST_10_FG3_PCT',
                                             'LAST_10_AST',
                                             'LAST_10_REB',])

#iterate once for every unique team (test size is large enough that it contains all teams)
for team in recent_stats_df['HOME_TEAM_ID'].unique():
  #get only the rows of the dataframe that contain the team in question
  team_df = recent_stats_df[(recent_stats_df['HOME_TEAM_ID'] == team) | (recent_stats_df['VISITOR_TEAM_ID'] == team)].sort_values(by='GAME_DATE_EST').tail(1).iloc[0]
  
  #store values for both home and away team
  h_team = team_df['HOME_TEAM_ID']
  a_team = team_df['VISITOR_TEAM_ID']
  h_last_3_pts = team_df['H_LAST_3_PTS']
  h_last_3_fg_pct = team_df['H_LAST_3_FG_PCT']
  h_last_3_ft_pct = team_df['H_LAST_3_FT_PCT']
  h_last_3_fg3_pct = team_df['H_LAST_3_FG3_PCT']
  h_last_3_ast = team_df['H_LAST_3_AST']
  h_last_3_reb = team_df['H_LAST_3_REB']
  h_last_5_pts = team_df['H_LAST_5_PTS']
  h_last_5_fg_pct = team_df['H_LAST_5_FG_PCT']
  h_last_5_ft_pct = team_df['H_LAST_5_FT_PCT']
  h_last_5_fg3_pct = team_df['H_LAST_5_FG3_PCT']
  h_last_5_ast = team_df['H_LAST_5_AST']
  h_last_5_reb = team_df['H_LAST_5_REB']
  h_last_10_pts = team_df['H_LAST_10_PTS']
  h_last_10_fg_pct = team_df['H_LAST_10_FG_PCT']
  h_last_10_ft_pct = team_df['H_LAST_10_FT_PCT']
  h_last_10_fg3_pct = team_df['H_LAST_10_FG3_PCT']
  h_last_10_ast = team_df['H_LAST_10_AST']
  h_last_10_reb = team_df['H_LAST_10_REB']
  a_last_3_pts = team_df['A_LAST_3_PTS']
  a_last_3_fg_pct = team_df['A_LAST_3_FG_PCT']
  a_last_3_ft_pct = team_df['A_LAST_3_FT_PCT']
  a_last_3_fg3_pct = team_df['A_LAST_3_FG3_PCT']
  a_last_3_ast = team_df['A_LAST_3_AST']
  a_last_3_reb = team_df['A_LAST_3_REB']
  a_last_5_pts = team_df['A_LAST_5_PTS']
  a_last_5_fg_pct = team_df['A_LAST_5_FG_PCT']
  a_last_5_ft_pct = team_df['A_LAST_5_FT_PCT']
  a_last_5_fg3_pct = team_df['A_LAST_5_FG3_PCT']
  a_last_5_ast = team_df['A_LAST_5_AST']
  a_last_5_reb = team_df['A_LAST_5_REB']
  a_last_10_pts = team_df['A_LAST_10_PTS']
  a_last_10_fg_pct = team_df['A_LAST_10_FG_PCT']
  a_last_10_ft_pct = team_df['A_LAST_10_FT_PCT']
  a_last_10_fg3_pct = team_df['A_LAST_10_FG3_PCT']
  a_last_10_ast = team_df['A_LAST_10_AST']
  a_last_10_reb = team_df['A_LAST_10_REB']

  #if team was home, take the home team values
  #otherwise take the away team values
  if team == h_team:
    last_3_pts = h_last_3_pts
    last_3_fg_pct = h_last_3_fg_pct
    last_3_ft_pct = h_last_3_ft_pct
    last_3_fg3_pct = h_last_3_fg3_pct
    last_3_ast = h_last_3_ast
    last_3_reb = h_last_3_reb

    last_5_pts = h_last_5_pts
    last_5_fg_pct = h_last_5_fg_pct
    last_5_ft_pct = h_last_5_ft_pct
    last_5_fg3_pct = h_last_5_fg3_pct
    last_5_ast = h_last_5_ast
    last_5_reb = h_last_5_reb

    last_10_pts = h_last_10_pts
    last_10_fg_pct = h_last_10_fg_pct
    last_10_ft_pct = h_last_10_ft_pct
    last_10_fg3_pct = h_last_10_fg3_pct
    last_10_ast = h_last_10_ast
    last_10_reb = h_last_10_reb

  else:
    last_3_pts = a_last_3_pts
    last_3_fg_pct = a_last_3_fg_pct
    last_3_ft_pct = a_last_3_ft_pct
    last_3_fg3_pct = a_last_3_fg3_pct
    last_3_ast = a_last_3_ast
    last_3_reb = a_last_3_reb

    last_5_pts = a_last_5_pts
    last_5_fg_pct = a_last_5_fg_pct
    last_5_ft_pct = a_last_5_ft_pct
    last_5_fg3_pct = a_last_5_fg3_pct
    last_5_ast = a_last_5_ast
    last_5_reb = a_last_5_reb

    last_10_pts = a_last_10_pts
    last_10_fg_pct = a_last_10_fg_pct
    last_10_ft_pct = a_last_10_ft_pct
    last_10_fg3_pct = a_last_10_fg3_pct
    last_10_ast = a_last_10_ast
    last_10_reb = a_last_10_reb

  #generate new row and append to df
  team_recent_stats_df_row = dict(TEAM = team,
                                  LAST_3_PTS = last_3_pts,
                                  LAST_3_FG_PCT = last_3_fg_pct,
                                  LAST_3_FT_PCT = last_3_ft_pct,
                                  LAST_3_FG3_PCT = last_3_fg3_pct,
                                  LAST_3_AST = last_3_ast,
                                  LAST_3_REB = last_3_reb,
                                  LAST_5_PTS = last_5_pts,
                                  LAST_5_FG_PCT = last_5_fg_pct,
                                  LAST_5_FT_PCT = last_5_ft_pct,
                                  LAST_5_FG3_PCT = last_5_fg3_pct,
                                  LAST_5_AST = last_5_ast,
                                  LAST_5_REB = last_5_reb,
                                  LAST_10_PTS = last_10_pts,
                                  LAST_10_FG_PCT = last_10_fg_pct,
                                  LAST_10_FT_PCT = last_10_ft_pct,
                                  LAST_10_FG3_PCT = last_10_fg3_pct,
                                  LAST_10_AST = last_10_ast,
                                  LAST_10_REB = last_10_reb,)

  team_recent_stats_df = team_recent_stats_df.append(team_recent_stats_df_row, ignore_index=True)

In [ ]:
#save to csv
team_recent_stats_df.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Team_Recent_Stats.csv', index=False)

# Add Engineered Columns

In [ ]:
#load all datasets
elo = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/ELO.csv')
team_elo = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Team_ELO.csv')
recent_stats = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Recent_Stats.csv')
team_recent_stats = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/Team_Recent_Stats.csv')

games_src_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/games_train.csv', index_col=0)


games_src_test = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/games_test.csv', index_col=0)


In [ ]:
#drop raw columns so we can replace with engineered columns
games_src_train = games_src_train.iloc[:, [0,1,2,3,4,17]]
games_src_train

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
0,2003-10-05,10300001,1610612762,1610612742,2003,1
1,2003-10-06,10300002,1610612763,1610612749,2003,1
2,2003-10-07,10300010,1610612764,1610612752,2003,1
3,2003-10-07,10300009,1610612758,1610612746,2003,1
4,2003-10-07,10300005,1610612757,1610612745,2003,1
...,...,...,...,...,...,...
23309,2020-08-22,41900103,1610612753,1610612749,2019,0
23310,2020-08-22,41900143,1610612757,1610612747,2019,0
23311,2020-08-22,41900133,1610612748,1610612754,2019,1
23312,2020-08-22,41900173,1610612760,1610612745,2019,1


In [ ]:
len(games_src_train), len(elo), len(recent_stats), len(final_df_train)

(23314, 23314, 23314, 23314)

In [ ]:
#generate new df and add all engineered columns
final_df_train = games_src_train.merge(elo.iloc[:, [1,4,5]], on='GAME_ID')
final_df_train = final_df_train.merge(recent_stats.iloc[:, [1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]], on='GAME_ID')
final_df_train.dropna(inplace=True) #drop the rows that did not have sufficient data to calculate recent stats (they have NAN)
final_df_train.reset_index(drop=True, inplace=True)

In [ ]:
#split X and y
final_df_y_train = final_df_train['HOME_TEAM_WINS']
final_df_X_train = final_df_train.drop(columns=['HOME_TEAM_WINS'])
final_df_y_test = games_src_test['HOME_TEAM_WINS']
final_df_X_test = games_src_test.iloc[:, :5]

In [ ]:
#save to csv
final_df_X_test.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Test.csv')
final_df_X_train.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Train.csv')
final_df_y_test.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Test.csv')
final_df_y_train.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Train.csv')

In [ ]:
X_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Train.csv', index_col=0)
X_test = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Test.csv', index_col=0)
y_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Train.csv', index_col=0)
y_test = pd.read_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Test.csv', index_col=0)

In [ ]:
#add engineered columns to test set
test_h_elo, test_a_elo = [],[]
test_h_last_3_pts, test_a_last_3_pts = [],[]
test_h_last_3_fg_pct, test_a_last_3_fg_pct = [],[]
test_h_last_3_ft_pct, test_a_last_3_ft_pct = [],[]
test_h_last_3_fg3_pct, test_a_last_3_fg3_pct = [],[]
test_h_last_3_ast, test_a_last_3_ast = [],[]
test_h_last_3_reb, test_a_last_3_reb = [],[]

test_h_last_5_pts, test_a_last_5_pts = [],[]
test_h_last_5_fg_pct, test_a_last_5_fg_pct = [],[]
test_h_last_5_ft_pct, test_a_last_5_ft_pct = [],[]
test_h_last_5_fg3_pct, test_a_last_5_fg3_pct = [],[]
test_h_last_5_ast, test_a_last_5_ast = [],[]
test_h_last_5_reb, test_a_last_5_reb = [],[]

test_h_last_10_pts, test_a_last_10_pts = [],[]
test_h_last_10_fg_pct, test_a_last_10_fg_pct = [],[]
test_h_last_10_ft_pct, test_a_last_10_ft_pct = [],[]
test_h_last_10_fg3_pct, test_a_last_10_fg3_pct = [],[]
test_h_last_10_ast, test_a_last_10_ast = [],[]
test_h_last_10_reb, test_a_last_10_reb = [],[]

for idx, row in X_test.iterrows():
  h_team = row['HOME_TEAM_ID']
  a_team = row["VISITOR_TEAM_ID"]
  
  h_elo = team_elo[team_elo['TEAM'] == h_team].iloc[0]['ELO']
  a_elo = team_elo[team_elo['TEAM'] == a_team].iloc[0]['ELO']
  
  h_last = team_recent_stats[team_recent_stats['TEAM'] == h_team].iloc[0]
  h_last_3_pts = h_last['LAST_3_PTS']
  h_last_3_fg_pct = h_last['LAST_3_FG_PCT']
  h_last_3_ft_pct = h_last['LAST_3_FT_PCT']
  h_last_3_fg3_pct = h_last['LAST_3_FG3_PCT']
  h_last_3_ast = h_last['LAST_3_AST']
  h_last_3_reb = h_last['LAST_3_REB']
  h_last_5_pts = h_last['LAST_5_PTS']
  h_last_5_fg_pct = h_last['LAST_5_FG_PCT']
  h_last_5_ft_pct = h_last['LAST_5_FT_PCT']
  h_last_5_fg3_pct = h_last['LAST_5_FG3_PCT']
  h_last_5_ast = h_last['LAST_5_AST']
  h_last_5_reb = h_last['LAST_5_REB']
  h_last_10_pts = h_last['LAST_10_PTS']
  h_last_10_fg_pct = h_last['LAST_10_FG_PCT']
  h_last_10_ft_pct = h_last['LAST_10_FT_PCT']
  h_last_10_fg3_pct = h_last['LAST_10_FG3_PCT']
  h_last_10_ast = h_last['LAST_10_AST']
  h_last_10_reb = h_last['LAST_10_REB']

  a_last = team_recent_stats[team_recent_stats['TEAM'] == a_team].iloc[0]
  a_last_3_pts = a_last['LAST_3_PTS']
  a_last_3_fg_pct = a_last['LAST_3_FG_PCT']
  a_last_3_ft_pct = a_last['LAST_3_FT_PCT']
  a_last_3_fg3_pct = a_last['LAST_3_FG3_PCT']
  a_last_3_ast = a_last['LAST_3_AST']
  a_last_3_reb = a_last['LAST_3_REB']
  a_last_5_pts = a_last['LAST_5_PTS']
  a_last_5_fg_pct = a_last['LAST_5_FG_PCT']
  a_last_5_ft_pct = a_last['LAST_5_FT_PCT']
  a_last_5_fg3_pct = a_last['LAST_5_FG3_PCT']
  a_last_5_ast = a_last['LAST_5_AST']
  a_last_5_reb = a_last['LAST_5_REB']
  a_last_10_pts = a_last['LAST_10_PTS']
  a_last_10_fg_pct = a_last['LAST_10_FG_PCT']
  a_last_10_ft_pct = a_last['LAST_10_FT_PCT']
  a_last_10_fg3_pct = a_last['LAST_10_FG3_PCT']
  a_last_10_ast = a_last['LAST_10_AST']
  a_last_10_reb = a_last['LAST_10_REB']

  
  test_h_elo.append(h_elo)
  test_h_last_3_pts.append(h_last_3_pts)
  test_h_last_3_fg_pct.append(h_last_3_fg_pct)
  test_h_last_3_ft_pct.append(h_last_3_ft_pct)
  test_h_last_3_fg3_pct.append(h_last_3_fg3_pct)
  test_h_last_3_ast.append(h_last_3_ast)
  test_h_last_3_reb.append(h_last_3_reb)
  test_h_last_5_pts.append(h_last_5_pts)
  test_h_last_5_fg_pct.append(h_last_5_fg_pct)
  test_h_last_5_ft_pct.append(h_last_5_ft_pct)
  test_h_last_5_fg3_pct.append(h_last_5_fg3_pct)
  test_h_last_5_ast.append(h_last_5_ast)
  test_h_last_5_reb.append(h_last_5_reb)
  test_h_last_10_pts.append(h_last_10_pts)
  test_h_last_10_fg_pct.append(h_last_10_fg_pct)
  test_h_last_10_ft_pct.append(h_last_10_ft_pct)
  test_h_last_10_fg3_pct.append(h_last_10_fg3_pct)
  test_h_last_10_ast.append(h_last_10_ast)
  test_h_last_10_reb.append(h_last_10_reb)

  test_a_elo.append(h_elo)
  test_a_last_3_pts.append(a_last_3_pts)
  test_a_last_3_fg_pct.append(a_last_3_fg_pct)
  test_a_last_3_ft_pct.append(a_last_3_ft_pct)
  test_a_last_3_fg3_pct.append(a_last_3_fg3_pct)
  test_a_last_3_ast.append(a_last_3_ast)
  test_a_last_3_reb.append(a_last_3_reb)
  test_a_last_5_pts.append(a_last_5_pts)
  test_a_last_5_fg_pct.append(a_last_5_fg_pct)
  test_a_last_5_ft_pct.append(a_last_5_ft_pct)
  test_a_last_5_fg3_pct.append(a_last_5_fg3_pct)
  test_a_last_5_ast.append(a_last_5_ast)
  test_a_last_5_reb.append(a_last_5_reb)
  test_a_last_10_pts.append(a_last_10_pts)
  test_a_last_10_fg_pct.append(a_last_10_fg_pct)
  test_a_last_10_ft_pct.append(a_last_10_ft_pct)
  test_a_last_10_fg3_pct.append(a_last_10_fg3_pct)
  test_a_last_10_ast.append(a_last_10_ast)
  test_a_last_10_reb.append(a_last_10_reb)



X_test['H_ELO'] = test_h_elo
X_test['A_ELO'] = test_a_elo
X_test['H_LAST_3_PTS'] = test_h_last_3_pts
X_test['A_LAST_3_PTS'] = test_a_last_3_pts
X_test['H_LAST_3_FG_PCT'] = test_h_last_3_fg_pct
X_test['A_LAST_3_FG_PCT'] = test_a_last_3_fg_pct
X_test['H_LAST_3_FT_PCT'] = test_h_last_3_ft_pct
X_test['A_LAST_3_FT_PCT'] = test_a_last_3_ft_pct
X_test['H_LAST_3_FG3_PCT'] = test_h_last_3_fg3_pct
X_test['A_LAST_3_FG3_PCT'] = test_a_last_3_fg3_pct
X_test['H_LAST_3_AST'] = test_h_last_3_ast
X_test['A_LAST_3_AST'] = test_a_last_3_ast
X_test['H_LAST_3_REB'] = test_h_last_3_reb
X_test['A_LAST_3_REB'] = test_a_last_3_reb
X_test['H_LAST_5_PTS'] = test_h_last_5_pts
X_test['A_LAST_5_PTS'] = test_a_last_5_pts
X_test['H_LAST_5_FG_PCT'] = test_h_last_5_fg_pct
X_test['A_LAST_5_FG_PCT'] = test_a_last_5_fg_pct
X_test['H_LAST_5_FT_PCT'] = test_h_last_5_ft_pct
X_test['A_LAST_5_FT_PCT'] = test_a_last_5_ft_pct
X_test['H_LAST_5_FG3_PCT'] = test_h_last_5_fg3_pct
X_test['A_LAST_5_FG3_PCT'] = test_a_last_5_fg3_pct
X_test['H_LAST_5_AST'] = test_h_last_5_ast
X_test['A_LAST_5_AST'] = test_a_last_5_ast
X_test['H_LAST_5_REB'] = test_h_last_5_reb
X_test['A_LAST_5_REB'] = test_a_last_5_reb
X_test['H_LAST_10_PTS'] = test_h_last_10_pts
X_test['A_LAST_10_PTS'] = test_a_last_10_pts
X_test['H_LAST_10_FG_PCT'] = test_h_last_10_fg_pct
X_test['A_LAST_10_FG_PCT'] = test_a_last_10_fg_pct
X_test['H_LAST_10_FT_PCT'] = test_h_last_10_ft_pct
X_test['A_LAST_10_FT_PCT'] = test_a_last_10_ft_pct
X_test['H_LAST_10_FG3_PCT'] = test_h_last_10_fg3_pct
X_test['A_LAST_10_FG3_PCT'] = test_a_last_10_fg3_pct
X_test['H_LAST_10_AST'] = test_h_last_10_ast
X_test['A_LAST_10_AST'] = test_a_last_10_ast
X_test['H_LAST_10_REB'] = test_h_last_10_reb
X_test['A_LAST_10_REB'] = test_a_last_10_reb

In [ ]:
#save to csv
X_train.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Train_Eng.csv')
X_test.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/X_Test_Eng.csv')
y_train.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Train_Eng.csv')
y_test.to_csv('drive/MyDrive/Colab Notebooks/Final/V2/Data/y_Test_Eng.csv')